In [55]:
!pip install yfinance numpy

In [56]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

# Section 1

## Q1

In [65]:
np.random.seed(0)

X = np.random.normal(0, 1, 5000)
Y = np.random.normal(0, 1, 5000)

rho = 0.5

U = X
V = rho * X + np.sqrt(1 - rho**2) * Y

# Calculate Correlation
correlation_matrix = np.corrcoef(U, V)
print("Correlation between U and V:", correlation_matrix[0, 1])

Correlation between U and V: 0.49929725923990254


## Q2

In [67]:
sp500_ticker = '^GSPC'  # S&P 500 index
usdcad_ticker = 'CAD=X'  # USD/CAD FX rate
start_date = '2019-12-31'
end_date = '2023-12-31'

sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date)
usdcad_data = yf.download(usdcad_ticker, start=start_date, end=end_date)

sp500_close = sp500_data['Close']
sp500_close= sp500_data['Close'].pct_change().dropna()
usdcad_close = usdcad_data['Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [68]:
merged_df = pd.merge(sp500_close, usdcad_close, on='Date', how='inner')
merged_df = merged_df.rename(columns={'Close_x': 'SP500_close', 'Close_y': 'usdcad_close'})
merged_df.head()

,SP500_close,usdcad_close
Date,,
2020-01-02,0.008379,1.29730
2020-01-03,-0.007060,1.29830
2020-01-06,0.003533,1.29866
2020-01-07,-0.002803,1.29639
2020-01-08,0.004902,1.30031


In [69]:
original_corr = np.corrcoef(merged_df["SP500_close"], merged_df["usdcad_close"])[0, 1]

rho_target = 0.5

sp500_adjusted = merged_df["SP500_close"]
usdcad_adjusted = rho_target * merged_df["SP500_close"] + np.sqrt(1 - rho_target**2) * (merged_df["usdcad_close"] - np.mean(merged_df["usdcad_close"])) / np.std(merged_df["usdcad_close"])

# Calculate the new correlation
new_corr = np.corrcoef(sp500_adjusted, usdcad_adjusted)[0, 1]

print(f"Original Correlation: {original_corr}")
print(f"New Correlation: {new_corr}")

Original Correlation: 0.04759361858074034
New Correlation: 0.05593221243023131


In [70]:
std_dev = merged_df['SP500_close'].std()

# Assuming 252 trading days in a year
annualized_volatility = std_dev * np.sqrt(252)

print(f"Annualized Volatility: {annualized_volatility * 100:.2f}%")

Annualized Volatility: 23.01%


## Q3

In [ ]:
np.random.seed(0)

# Define correlation matrix
correlation_matrix = np.array([
    [1.0, 0.8, 0.6, 0.4],
    [0.8, 1.0, 0.7, 0.5],
    [0.6, 0.7, 1.0, 0.9],
    [0.4, 0.5, 0.9, 1.0]
])

data = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=correlation_matrix, size=5000).T

# Adjust correlations to desired values
desired_correlation_matrix = np.array([
    [1.0, 0.5, 0.5, 0.5],
    [0.5, 1.0, 0.5, 0.5],
    [0.5, 0.5, 1.0, 0.5],
    [0.5, 0.5, 0.5, 1.0]
])

# Perform Cholesky decomposition of the current correlation matrix
cholesky_decomposition = np.linalg.cholesky(correlation_matrix)

# Scale the decomposition to achieve the desired correlations
adjusted_cholesky = cholesky_decomposition @ np.linalg.inv(np.linalg.cholesky(desired_correlation_matrix))

adjusted_data = adjusted_cholesky @ data

adjusted_data1 = adjusted_data[0]
adjusted_data2 = adjusted_data[1]
adjusted_data3 = adjusted_data[2]
adjusted_data4 = adjusted_data[3]

# Calculate the adjusted correlations
adjusted_correlation_matrix = np.corrcoef(adjusted_data)
print("Adjusted Correlation Matrix:")
print(adjusted_correlation_matrix)


Adjusted Correlation Matrix:
[[1.         0.92328455 0.70460917 0.36313474]
 [0.92328455 1.         0.79361637 0.46023321]
 [0.70460917 0.79361637 1.         0.89465254]
 [0.36313474 0.46023321 0.89465254 1.        ]]


# Section 3

## Q5

Use piecewise aggregate approximation (PAA) to resample the time series to a common frequency.

In [ ]:
# Sample time series data
time_series1 = {
    'Date': ['2019-12-31', '2020-01-02', '2020-01-03', '2020-01-06'],
    'Close1': [3230.78, 3257.85, 3234.85, 3246.28]
}

time_series2 = {
    'Date': ['2019-12-31', '2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06'],
    'Close': [1.30606, 1.3002, 1.2973, 1.2983, 1.29866]
}


df1 = pd.DataFrame(time_series1)
df2 = pd.DataFrame(time_series2)

df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

df1.set_index('Date', inplace=True)
df2.set_index('Date', inplace=True)

# Resample both time series to daily frequency (filling missing values with NaN)
df1_resampled = df1.resample('D').mean()
df2_resampled = df2.resample('D').mean()

df1_resampled.interpolate(method='linear', inplace=True)
df2_resampled.interpolate(method='linear', inplace=True)

common_dates = df1_resampled.index.intersection(df2_resampled.index)
aligned_df1 = df1_resampled.loc[common_dates]
aligned_df2 = df2_resampled.loc[common_dates]

print(aligned_df1)
print(aligned_df2)


              Close1
Date                
2019-12-31  3230.780
2020-01-01  3244.315
2020-01-02  3257.850
2020-01-03  3234.850
2020-01-04  3238.660
2020-01-05  3242.470
2020-01-06  3246.280
              Close
Date               
2019-12-31  1.30606
2020-01-01  1.30020
2020-01-02  1.29730
2020-01-03  1.29830
2020-01-04  1.29842
2020-01-05  1.29854
2020-01-06  1.29866


# Section 4

## Q8

In [48]:
# Fetching historical data for the assets
def fetch_data(ticker):
    data = yf.download(tickers=ticker, start="2022-01-01", end = "2023-01-01",
period='1d')
    return data['Adj Close']

# Calculate daily returns
def calculate_returns(data):
    returns = data.pct_change().dropna()
    return returns

def calculate_VaR(df):
    results = {}
    for column in df.columns:
        sorted_col = df[column].sort_values()
        n = int(len(sorted_col) * 0.1)
        sum_lowest = sorted_col.iloc[:n].mean()
        results[column] = sum_lowest
    return results


def calculate_contribution_VaR(df):
  sorted_df = df.sort_values(by='Sum')

  num_rows = int(len(sorted_df) * 0.1)

  top_10_percent = sorted_df.iloc[:num_rows]

  average_values = top_10_percent.mean()

  return average_values


**Assumptions**:
1. I used one year data as an assumption of one-day data to calculate the 1 Day VaR.
2. I used daily return as the PnLs for calculation.
**Reason**: Yahoo Finance is not available to extract one day with a frequency of 1 minute or 5 minutes of historical date. It can only do the current date. But the data is not enough since we are only at noon. So I just do 1 year data to show my calculation process.

In [88]:
# S&P500, S&P/TSX indices, S&P GSCI Index, Gold, US 10Y Treasuries and CA 10Y Treasuries.
tickers = ['^GSPC','^GSPTSE','GD=F','GC=F', 'IEF', 'XGB.TO']


data = fetch_data(tickers)

usdcad_close.index = pd.to_datetime(usdcad_close.index)

FX_rate= usdcad_close['2022-01-01':'2023-01-01']

FX_rate=FX_rate.to_frame()

merge_data=pd.merge(data,FX_rate,on='Date',how='inner')

merge_data['GC=F']=merge_data['GC=F']* merge_data['Close']
merge_data['GD=F']=merge_data['GD=F']* merge_data['Close']
merge_data['IEF']=merge_data['IEF']* merge_data['Close']
merge_data['^GSPC']=merge_data['^GSPC']* merge_data['Close']
merge_data['^GSPTSE']=merge_data['^GSPTSE']* merge_data['Close']

merge_data=merge_data.drop(columns=['Close'])

returns = calculate_returns(merge_data)

returns['Sum'] = returns.sum(axis=1)

#Calculate VaR
results_VaR = calculate_VaR(returns)

# Calculate Contribution VaR
results_contribution=calculate_contribution_VaR(returns)

# Caluclate incremental VaR
row_differences = returns.diff().iloc[1:]
results_incre=calculate_VaR(row_differences)

[*********************100%%**********************]  6 of 6 completed


In [89]:
results_VaR

{'GC=F': -0.016959014719222947,
 'GD=F': -0.03647053218296761,
 'IEF': -0.013963447166142959,
 'XGB.TO': -0.00907006930665195,
 '^GSPC': -0.028475269772291745,
 '^GSPTSE': -0.02028779146161965,
 'Sum': -0.08371752735035053}

In [90]:
results_contribution

GC=F      -0.010328
GD=F      -0.020991
IEF       -0.007020
XGB.TO    -0.003352
^GSPC     -0.023388
^GSPTSE   -0.018638
Sum       -0.083718
dtype: float64

In [91]:
results_incre

{'GC=F': -0.028674418901720792,
 'GD=F': -0.049792872028071915,
 'IEF': -0.021071625698782393,
 'XGB.TO': -0.012895782551186815,
 '^GSPC': -0.04532839539890153,
 '^GSPTSE': -0.030588606857617724,
 'Sum': -0.13114699652814427}